In [ ]:
# Lets start by creating a PDF image

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from nanopyx.core.particles.simulate_particle_field import (
    get_closest_distance, render_particle_gaussians_with_tracks,
    render_particle_histogram, render_particle_histogram_with_tracks,
    simulate_particle_field_based_on_2D_PDF)

x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, y)
image_pdf = np.exp(-(X - 0.5)**2 / 0.1**2 - (Y - 0.5)**2 / 0.1**2)
image_pdf = image_pdf.astype(np.float32)

# Simulate the particle field
particle_field, mean_closest_distance = simulate_particle_field_based_on_2D_PDF(
    image_pdf, min_particles=100, max_particles=1E3, min_distance=0.01, mean_distance_threshold=0.1)

print(f"Number of particles: {particle_field.shape[0]}")
print(f"Mean closest distance: {mean_closest_distance}")
print(f"Closest distance: {get_closest_distance(particle_field)}")

image_particle_field = render_particle_histogram(particle_field, 100, 100)

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
# Create the first heatmap
sns.heatmap(image_pdf, cmap='inferno', square=True, ax=ax1)
ax1.set_title("PDF")
# Create the second heatmap
sns.heatmap(image_particle_field, cmap='inferno', square=True, ax=ax2)
ax2.set_title("Particle histogram")
# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.1)
# Display the figure
plt.show()

In [ ]:
# Now lets render the time tracks of the particles

from nanopyx.core.particles.simulate_photoswitching_time_tracks import simple_state_transition_model

# Define the parameters
n_particles = particle_field.shape[0]
n_steps = 1000
p_on = 0.01
p_transient_off = 0.1
p_permanent_off = 0.01

# Simulate the particle field
particle_tracks = simple_state_transition_model(n_particles, n_steps, p_on, p_transient_off, p_permanent_off, 1)

sns.heatmap(particle_tracks, cmap='inferno', square=False)


In [ ]:

# Render the particle tracks as a movie

# image_stack = render_particle_histogram_with_tracks(particle_field, particle_tracks, 100, 100)
image_stack = render_particle_gaussians_with_tracks(particle_field, particle_tracks, 100, 100, 1000, 1.5, 1.5)

import stackview
stackview.slice(image_stack, continuous_update=True, zoom_factor=4, zoom_spline_order=0)

In [ ]:
# from celluloid import Camera
# from IPython.display import HTML

# fig, ax = plt.subplots() # make it bigger
# camera = Camera(fig)
# for i in range(image_stack.shape[0]):
#     ax.imshow(image_stack[i], cmap='inferno')
#     camera.snap()
# animation = camera.animate()
# HTML(animation.to_html5_video())